In [10]:
import pandas as pd
import numpy as np

# Reading and Writing Data in Text Format

In [7]:
#we can use read_csv to read it into a DataFrame
pd.read_csv('examples\ex1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
#We could also have used read_table and specified the delimiter
pd.read_table('examples/ex1.csv', sep=',')

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
#A file will not always have a header row
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [11]:
#you can specify names yourself
pd.read_csv('examples/ex2.csv',names=['a','b','c','d','mess'])

,a,b,c,d,mess
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
names = ['a', 'b', 'c', 'd', 'message']

In [13]:
#making message column as an index for dataframe
pd.read_csv('examples/ex2.csv',names=names,index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [15]:
#hierarchical index
parsed = pd.read_csv('examples/csv_mindex.csv',index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [16]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [17]:
#passing regular expression as a delimiter
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [18]:
#skipping rows
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [19]:
#Handling missing values
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [21]:
pd.read_csv('examples/ex5.csv',na_values=['Null'])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
#Different NA sentinels can be specified for each column in a dict
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv',na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## Reading Text Files in Pieces

In [23]:
#Before we look at a large file, we make the pandas display settings more compact
pd.options.display.max_rows = 10

In [24]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [25]:
#To read a small number of rows
pd.read_csv('examples/ex6.csv',nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [26]:
#To read a file in pieces, specify a chunksize as a number of rows
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [27]:
#we can iterate over ex6.csv, aggregating the value counts in the 'key' column
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

In [28]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

In [31]:
result.key.value_counts()

E    368
X    364
L    346
O    343
Q    340
    ... 
5    157
2    152
0    151
9    150
1    146
Name: key, Length: 36, dtype: int64

In [32]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## Writing Data to Text Format

In [33]:
#Data can also be exported to a delimited format
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [34]:
#Using DataFrame’s to_csv method, we can write the data out to a comma-separated file
data.to_csv('example/out.csv')

In [35]:
#writing to sys.stdout so it prints the text result to the console
import sys
#other delimiter
data.to_csv(sys.stdout,sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [36]:
#Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value
data.to_csv(sys.stdout,na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [37]:
#removing row labels
data.to_csv(sys.stdout,index=False)

something,a,b,c,d,message
one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [38]:
#removing column labels
data.to_csv(sys.stdout,header=False)

0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [39]:
#removing both row and column labels
data.to_csv(sys.stdout,index=False,header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [40]:
#write only a subset of the columns
data.to_csv(sys.stdout,columns=['a','b','c'])

,a,b,c
0,1,2,3.0
1,5,6,
2,9,10,11.0


In [42]:
#Series also has a to_csv method
dates = pd.date_range('01/01/2020',periods=7)
dates

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07'],
              dtype='datetime64[ns]', freq='D')

In [45]:
import numpy as np
ts = pd.Series(np.arange(7),index=dates)
ts

2020-01-01    0
2020-01-02    1
2020-01-03    2
2020-01-04    3
2020-01-05    4
2020-01-06    5
2020-01-07    6
Freq: D, dtype: int32

In [46]:
ts.to_csv('example/tseries.csv')

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [47]:
pd.read_csv('example/tseries.csv')

,2020-01-01,0
0,2020-01-02,1
1,2020-01-03,2
2,2020-01-04,3
3,2020-01-05,4
4,2020-01-06,5
5,2020-01-07,6


## Working with Delimited Formats

In [49]:
#For any file with a single-character delimiter, you can use Python’s built-in csv module
import csv
f = open('examples/ex7.csv')
read = csv.reader(f)
read

In [50]:
#iterating through read to print lines
for line in read:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [51]:
#First, we read the file into a list of lines
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [52]:
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [53]:
#Then, we split the lines into the header line and the data lines
header,values = lines[0],lines[1:]

In [54]:
#zip(*values) to trnaspose rows to columns
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [55]:
data_dict1 = {h: v for h, v in zip(header, values)}
data_dict1

{'a': ['1', '2', '3'], 'b': ['1', '2', '3']}

In [56]:
#Defining our own format
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
reader = csv.reader(f, dialect=my_dialect)

TypeError: argument 1 must be an iterator

In [57]:
#To write delimited files manually, you can use csv.writer
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [58]:
pd.read_csv('mydata.csv')

,one;two;three
0,1;2;3
1,4;5;6
2,7;8;9


## JSON Data

In [59]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
{"name": "Katie", "age": 38,
"pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [60]:
obj

'\n{"name": "Wes",\n"places_lived": ["United States", "Spain", "Germany"],\n"pet": null,\n"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},\n{"name": "Katie", "age": 38,\n"pets": ["Sixes", "Stache", "Cisco"]}]\n}\n'

In [62]:
#To convert a JSON string to Python form, use json.loads
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [63]:
#json.dumps, on the other hand, converts a Python object back to JSON
asjson = json.dumps(result)

In [64]:
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [65]:
#giving subset of data to data frame loaded from json format
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [67]:
#The pandas.read_json can automatically convert JSON datasets in specific arrangements into a Series or DataFrame
data = pd.read_json('examples/example.json')
#The default options for pandas.read_json assume that each object in the JSON array is a row in the table
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [68]:
#using to_json for exporting data from python to json
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [69]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML and HTML: Web Scraping

In [70]:
conda install lxml

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\HP\Anaconda3

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.27 |                0         163 KB
    certifi-2019.11.28         |           py37_0         157 KB
    conda-4.8.1                |           py37_0         3.0 MB
    lxml-4.4.1                 |   py37h1350720_0         1.2 MB
    ------------------------------------------------------------
                                           Total:         4.6 MB

The following packages will be UPDATED:

  ca-certificates                              2019.10.16-0 --> 2019.11.27-0
  certifi                                  2019.9.11-py37_0 --> 2019.11.28-py37_0
  conda                                       4.7.12-py37_0 --> 4.8.1-py37_0
  lxml                                 4.3

In [8]:
!pip install beautifulsoup4 html5lib

In [2]:
#By default .read_html attempts to parse all tabular data contained within <table> tags
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [3]:
tables

[                                  Bank Name             City  ST   CERT  \
 0                               Allied Bank         Mulberry  AR     91   
 1              The Woodbury Banking Company         Woodbury  GA  11297   
 2                    First CornerStone Bank  King of Prussia  PA  35312   
 3                        Trust Company Bank          Memphis  TN   9956   
 4                North Milwaukee State Bank        Milwaukee  WI  20364   
 5                    Hometown National Bank         Longview  WA  35156   
 6                       The Bank of Georgia   Peachtree City  GA  35259   
 7                              Premier Bank           Denver  CO  34112   
 8                            Edgebrook Bank          Chicago  IL  57772   
 9                    Doral Bank  En Espanol         San Juan  PR  32102   
 10        Capitol City Bank & Trust Company          Atlanta  GA  33938   
 11                  Highland Community Bank          Chicago  IL  20290   
 12         

In [4]:
len(tables)

1

In [5]:
failures = tables[0]

In [6]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [7]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [8]:
close_timestamps

0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
5     2015-10-02
6     2015-10-02
7     2015-07-10
8     2015-05-08
9     2015-02-27
10    2015-02-13
11    2015-01-23
12    2015-01-16
13    2014-12-19
14    2014-11-07
15    2014-10-24
16    2014-10-17
17    2014-07-25
18    2014-07-18
19    2014-06-27
20    2014-06-20
21    2014-06-20
22    2014-05-30
23    2014-05-23
24    2014-05-16
25    2014-04-25
26    2014-02-28
27    2014-02-28
28    2014-01-31
29    2014-01-24
         ...    
517   2008-05-09
518   2008-03-07
519   2008-01-25
520   2007-10-04
521   2007-09-28
522   2007-02-02
523   2004-06-25
524   2004-03-19
525   2004-03-12
526   2004-02-14
527   2003-11-14
528   2003-05-09
529   2003-02-07
530   2002-12-17
531   2002-11-08
532   2002-09-30
533   2002-06-27
534   2002-06-26
535   2002-03-28
536   2002-03-01
537   2002-02-07
538   2002-02-01
539   2002-01-18
540   2002-01-11
541   2001-09-07
542   2001-07-27
543   2001-05-03
544   2001-02-

In [9]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

# Binary Data Formats

In [11]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [12]:
#Storing data in binary format
frame.to_pickle('example/frame_pickle')

In [13]:
#Reading data that is stored in binary format
pd.read_pickle('example/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Using HDF5 Format

In [11]:
frame = pd.DataFrame({'a':np.random.randn(100)})
frame

,a
0,1.922716
1,1.114794
2,-0.197129
3,0.929809
4,0.420689
5,0.900672
6,2.407205
7,0.231881
8,-0.878078
9,0.182994


In [12]:
#creates .h5 file
store = pd.HDFStore('mydata.h5')
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [13]:
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [14]:
#Objects contained in the HDF5 file can then be retrieved with the same dict-like API
store['obj1']

,a
0,1.922716
1,1.114794
2,-0.197129
3,0.929809
4,0.420689
5,0.900672
6,2.407205
7,0.231881
8,-0.878078
9,0.182994


In [15]:
#HDFStore supports two storage schemas, 'fixed' and 'table'
store.put('obj2',frame,format='table')

In [16]:
store.put('obj3',frame,format='fixed')

In [18]:
store.select('obj2',where=['index >=10 and index<=15'])

,a
10,-1.291068
11,1.511253
12,-1.161215
13,0.873911
14,0.150832
15,0.841680


In [28]:
store.select('obj3')

,a
0,1.922716
1,1.114794
2,-0.197129
3,0.929809
4,0.420689
5,0.900672
6,2.407205
7,0.231881
8,-0.878078
9,0.182994


In [27]:
#we cannot use where clause when data is stored in fixed format in HDF5
store.select('obj3',where=['index <10 and index>=0'])

TypeError: cannot pass a where specification when reading from a Fixed format store. this store must be selected in its entirety

In [29]:
store.close()

In [32]:
#we can use pandas.read function to read HDF5 data
pd.read_hdf('mydata.h5','obj2',where = ['index<10'])

,a
0,1.922716
1,1.114794
2,-0.197129
3,0.929809
4,0.420689
5,0.900672
6,2.407205
7,0.231881
8,-0.878078
9,0.182994


In [33]:
#we can store data in HDF5 by using pandas.to function
frame.to_hdf('mydata','obj4',format='table')

## Reading Microsoft Excel Files

In [14]:
#To use ExcelFile, create an instance by passing a path to an xls or xlsx file
xlsx = pd.ExcelFile('examples/ex1.xlsx')
xlsx

In [15]:
#Data stored in a sheet can then be read into DataFrame with parse
pd.read_excel(xlsx,'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [16]:
#We can aslo use pandas.read to read excel file
#but if we want yo read multiple sheets then it is better to first create ExcelFile instance and then read excel data
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [17]:
#To write pandas data to Excel format, you must first create an ExcelWriter, then
#write data to it using pandas objects’ to_excel method
writer = pd.ExcelWriter('example/ex2.xlsx')
frame.to_excel(writer,'Sheet1')
writer.save()

In [18]:
#You can also pass a file path to to_excel and avoid the ExcelWriter
frame.to_excel('examples/ex2.xlsx')

# Interacting with Web APIs

In [35]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
rep = requests.get(url)
rep

<Response [200]>

In [36]:
#The Response object’s json method will return a dictionary containing JSON parsed into native Python objects
data = rep.json()
data[0]['title']

'BLD: More lightweight mypy pre-commit hook'

In [37]:
#We can pass data directly to DataFrame and extract fields of interest
issues = pd.DataFrame(data, columns=['number', 'title','labels', 'state'])
issues

,number,title,labels,state
0,30814,BLD: More lightweight mypy pre-commit hook,[],open
1,30813,[DOC] add example of rolling with win_type gau...,[],open
2,30812,COMPAT: bump minimum version to pyarrow 0.13,"[{'id': 527603109, 'node_id': 'MDU6TGFiZWw1Mjc...",open
3,30811,BLD: Remove mypy from pre-commit as long its a...,"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNT...",open
4,30808,DOC: Move import conventions from wiki to docs,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
5,30806,REF: use shareable code for DTI/TDI.insert,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
6,30799,Tests for Deprecate SparseArray for python 3.6...,"[{'id': 76865106, 'node_id': 'MDU6TGFiZWw3Njg2...",open
7,30790,PERF: performance regression in 1.0 compared t...,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
8,30789,REF: Implement BaseMaskedArray class for integ...,"[{'id': 849023693, 'node_id': 'MDU6TGFiZWw4NDk...",open
9,30787,Unexpected behavior in cut() with nullable Int...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


# Interacting with Databases

In [39]:
import sqlite3

In [40]:
query = """
        CREATE TABLE test
        (a VARCHAR(20), b VARCHAR(20),
        c REAL, d INTEGER
        );"""

In [41]:
con = sqlite3.connect('mydata.sqlite')

In [43]:
con.execute(query)

In [44]:
con.commit()

In [45]:
#insert a few rows of data
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

In [46]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [47]:
con.executemany(stmt,data)

In [48]:
con.commit()

In [49]:
#Most Python SQL drivers eturn a list of tuples when selecting data from a table
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [51]:
#You can pass the list of tuples to the DataFrame constructor, but you also need the
#column names, contained in the cursor’s description attribute
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [52]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [53]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
